<a href="https://colab.research.google.com/github/agiagoulas/page-stream-segmentation/blob/master/model_training/CSV_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to Google Drive when working in Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Set working_dir

In [ ]:
working_dir = "/Tobacco800/" # TODO: Set correct working directory

Imports

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
import csv
from os import listdir
from os.path import isfile, join
from PIL import Image
import cv2
import pytesseract
import numpy as np

# CSV File Generation

OCR Extraction with Tessaract OCR

In [4]:
def parse_image_to_str(image_file):
    # open image
    img = cv2.imread(image_file)

    # parse to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray, img_bin = cv2.threshold(gray,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    gray = cv2.bitwise_not(img_bin)

    # ocr extraction
    kernel = np.ones((2, 1), np.uint8)
    img = cv2.erode(gray, kernel, iterations=1)
    img = cv2.dilate(img, kernel, iterations=1)
    out_below = pytesseract.image_to_string(img, lang="eng")
    return out_below

CSV File Generation

In [5]:
def create_csv_file(target_file, source_dir):
    file = open(target_file, 'a')

    source_files = sorted([f for f in listdir(source_dir) if isfile(join(source_dir, f))])

    with file:
        writer = csv.writer(file, csv.QUOTE_NONNUMERIC, delimiter=';')
        writer.writerow(["counter", "documentText", "label", "documentName"])
        
        past_file_title = ""
        for counter, file_name in enumerate(source_files):
            file_name_split = file_name.split('_')
            current_file_title = file_name_split[0]
            print(counter+1, "of", len(source_files))

            file_content = parse_image_to_str(source_dir + file_name)

            if past_file_title == current_file_title:
                writer.writerow([counter, file_content, "NextPage", file_name])
            else: 
                current_file_title = current_file_title.split('.')[0]
                writer.writerow([counter, file_content, "FirstPage", file_name])

            past_file_title = current_file_title

In [6]:
source_train_files = working_dir + "Tobacco800_Train/"
source_test_files = working_dir + "Tobacco800_Test/"
target_train_csv_file = working_dir + "tobacco800.train"
target_test_csv_file = working_dir + "tobacco800.test"

In [ ]:
create_csv_file(target_train_csv_file, source_train_files)

In [ ]:
create_csv_file(target_test_csv_file, source_test_files)